# Participant Locations
Looking at where the participants were during the UTx000 study.

In [111]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

# Participant Information

In [82]:
info = pd.read_excel("../data/raw/utx000/admin/s20c2_participant_list.xlsx",sheet_name="Complete")
info.dropna(subset=["Address"],inplace=True)
info.reset_index(inplace=True)
info = info[["Beiwe ID","First","Last","Address"]]
info[:15]

,Beiwe ID,First,Last,Address
0,y1tvkx14,Aaron,Alterman,"1701 Brook Hollow Dr, Fort Worth, TX, 76114"
1,pgvvwyvh,Aaron,Wheat,"2715 9th Street Wichita Falls, TX, 76301-3912"
2,idbkjh8u,Alana,Martinez,"1301 Crossing Pl apt 1513, Austin, TX 78741"
3,eyf8oqwl,Alexander,Chan,4918 Laurel St Bellaire TX 77401
4,789gcb6u,Adriana,Banchs,"3633 Aquamarine Dr. Round Rock, TX 78681"
5,kyj367pi,Angela,Gordillo,"2822 Rio Grande St. Apt 202 Austin, TX, 78705"
6,lkkjddam,Austin,Reynoso,806 East High St. Grand Saline TX 75140
7,8vpj3b9v,Chloe,Juanico,"20106 Cypresswood Glen, Spring, TX, 77373"
8,rnse61g4,Chloe,Roman,600 W. 26th St APT 3106 Austin TX 78705
9,wnprz3dq,Claire,Hemingway,"4606 Avenue F Austin, Tx 78751"


# Getting City and State
We can take advantage of address format to pull out the state and city that participants live in.

In [83]:
def add_city_state(df):
    """
    Adds the city and the state to the given dataframe
    """
    # pre-allocating
    df["state"] = "TX"
    df["city"] = "Austin"
    df.reset_index() # precaution
    for i in range(len(df)):
        address = df.loc[i,"Address"]
        state = address.split(" ")[-2].replace(",","").replace(".","").upper()
        city = address.split(" ")[-3].replace(",","").replace(".","").upper()
        df.loc[i,"state"] = state
        df.loc[i,"city"] = city

    df.replace("TEXAS","TX",inplace=True)

In [84]:
add_city_state(info)
info

,Beiwe ID,First,Last,Address,state,city
0,y1tvkx14,Aaron,Alterman,"1701 Brook Hollow Dr, Fort Worth, TX, 76114",TX,WORTH
1,pgvvwyvh,Aaron,Wheat,"2715 9th Street Wichita Falls, TX, 76301-3912",TX,FALLS
2,idbkjh8u,Alana,Martinez,"1301 Crossing Pl apt 1513, Austin, TX 78741",TX,AUSTIN
3,eyf8oqwl,Alexander,Chan,4918 Laurel St Bellaire TX 77401,TX,BELLAIRE
4,789gcb6u,Adriana,Banchs,"3633 Aquamarine Dr. Round Rock, TX 78681",TX,ROCK
...,...,...,...,...,...,...
62,rjcs3hyw,Samantha,Liu,"8125 Haning Drive Plano, TX 75025",TX,PLANO
63,1adkek2h,Victor,Mwaura,"2830 lake rd, Huntsville, TX, 77340",TX,HUNTSVILLE
64,y4m7yv2u,Megan,Ramos,"40 Newtown Ave North Kingstown, RI 02852",RI,KINGSTOWN
65,naucsx6v,Alvin,Ta,"5716 Sabetha Way, Plano, TX 75094",TX,PLANO


# Looking at the Results

In [85]:
print("Number in Texas:", len(info[info['state'] == 'TX']))
print("Number in Austin:", len(info[info['city'] == 'AUSTIN']))
print("Number Out-of-State:", len(info[info['state'] != 'TX']))

Number in Texas: 65
Number in Austin: 26
Number Out-of-State: 2


# Address Change
The addresses might have changed at some point during the study.

In [86]:
change = pd.read_excel("../data/raw/utx000/admin/s20c2_participant_list.xlsx",sheet_name="Kit Mailing")
change = change[["First","Last","Address"]]
change.head()

,First,Last,Address
0,Aaron,Alterman,"1701 Brook Hollow Dr, Fort Worth, TX, 76114"
1,Aaron,Wheat,"305 E. 21st St. #I003 Austin, TX 78705"
2,Alana,Martinez,"911 W 21st St Apt 2507, Austin, TX 78705"
3,Alexander,Chan,2529 Rio Grande Street Apt #89 Austin TX 78705
4,Angela,Gordillo,"4605 Avenue A APT 214, Austin, TX, 78751"


In [99]:
add_city_state(change)
comb = info.merge(right=change,on=["First","Last"],how="left",suffixes=["_start","_end"])
comb["moved"] = [True if s == e else False for s, e in zip(comb["city_start"], comb["city_end"])]
comb

,Beiwe ID,First,Last,Address_start,state_start,city_start,Address_end,state_end,city_end,moved
0,y1tvkx14,Aaron,Alterman,"1701 Brook Hollow Dr, Fort Worth, TX, 76114",TX,WORTH,"1701 Brook Hollow Dr, Fort Worth, TX, 76114",TX,WORTH,True
1,pgvvwyvh,Aaron,Wheat,"2715 9th Street Wichita Falls, TX, 76301-3912",TX,FALLS,"305 E. 21st St. #I003 Austin, TX 78705",TX,AUSTIN,False
2,idbkjh8u,Alana,Martinez,"1301 Crossing Pl apt 1513, Austin, TX 78741",TX,AUSTIN,"911 W 21st St Apt 2507, Austin, TX 78705",TX,AUSTIN,True
3,eyf8oqwl,Alexander,Chan,4918 Laurel St Bellaire TX 77401,TX,BELLAIRE,2529 Rio Grande Street Apt #89 Austin TX 78705,TX,AUSTIN,False
4,789gcb6u,Adriana,Banchs,"3633 Aquamarine Dr. Round Rock, TX 78681",TX,ROCK,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...
63,rjcs3hyw,Samantha,Liu,"8125 Haning Drive Plano, TX 75025",TX,PLANO,NaN,NaN,NaN,False
64,1adkek2h,Victor,Mwaura,"2830 lake rd, Huntsville, TX, 77340",TX,HUNTSVILLE,NaN,NaN,NaN,False
65,y4m7yv2u,Megan,Ramos,"40 Newtown Ave North Kingstown, RI 02852",RI,KINGSTOWN,NaN,NaN,NaN,False
66,naucsx6v,Alvin,Ta,"5716 Sabetha Way, Plano, TX 75094",TX,PLANO,NaN,NaN,NaN,False


In [108]:
to_save = comb.drop(["First","Last"],axis="columns").rename({"Beiwe ID":"beiwe","Address_start":"address_start","address_end":"address_end"},axis="columns")

In [110]:
to_save.to_csv("../data/raw/utx000/admin/participant_locations.csv")